In [2]:
spark

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
!hdfs dfs -ls

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
invalid syntax (<stdin>, line 1)
  File "<stdin>", line 1
    !hdfs dfs -ls
    ^
SyntaxError: invalid syntax



In [8]:
df = spark.read.csv('s3://aws-emr-resources-678153616337-us-east-1/olist_classified_public_dataset.csv', header=True, inferSchema=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- _c0: integer (nullable = true)
 |-- id: integer (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_products_value: double (nullable = true)
 |-- order_freight_value: double (nullable = true)
 |-- order_items_qty: integer (nullable = true)
 |-- order_sellers_qty: integer (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_aproved_at: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)
 |-- customer_zip_code_prefix: integer (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_name_lenght: integer (nullable = true)
 |-- product_description_lenght: integer (nullable = true)
 |-- product_photos_qty: integer (nullable = true)
 |-- review_score: integer (nullable = true)
 |-- review_comment_title: string

In [12]:
df.show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+---+------------+--------------------+-------------------+---------------+-----------------+------------------------+--------------------+-----------------------------+-----------------------------+---------------+--------------+------------------------+---------------------+-------------------+--------------------------+------------------+------------+--------------------+----------------------+--------------------+-----------------------+---------------------+-------------+-----------------+------------+---------------------+----------------------+--------------------+-----------------+---------------+-------------------+--------------------+
|_c0| id|order_status|order_products_value|order_freight_value|order_items_qty|order_sellers_qty|order_purchase_timestamp|    order_aproved_at|order_estimated_delivery_date|order_delivered_customer_date|  customer_city|customer_state|customer_zip_code_prefix|product_category_name|product_name_lenght|product_description_lenght|product_photos

In [20]:
from pyspark.sql.functions import *

df2 = df.withColumn("ts", to_timestamp(col("order_purchase_timestamp"))) \
    .withColumn("datetype", to_date(col("order_purchase_timestamp"),"yyyy-MM-dd"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
df2.select('datetype','ts').show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------------------+
|  datetype|                 ts|
+----------+-------------------+
|2017-08-30|2017-08-30 11:41:01|
|2017-09-26|2017-09-26 09:13:36|
|2018-01-15|2018-01-15 15:50:42|
|2018-02-04|2018-02-04 11:16:42|
|2017-12-07|2017-12-07 11:58:42|
+----------+-------------------+
only showing top 5 rows

In [33]:
df3 = df2.withColumn('week_of_month', date_format(col('ts'), 'W')) \
    .withColumn('day_of_week', dayofweek(col('ts')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
df3.select('week_of_month', 'day_of_week').show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+-----------+
|week_of_month|day_of_week|
+-------------+-----------+
|            5|          4|
|            5|          3|
|            3|          2|
|            2|          1|
|            2|          5|
|            1|          5|
|            5|          1|
|            2|          6|
|            2|          3|
|            5|          4|
+-------------+-----------+
only showing top 10 rows

In [43]:
#df3.printSchema()
df3.write.format('csv').option('header','true').save('s3://aws-emr-resources-678153616337-us-east-1/olist_classified_public_dataset_1')

#'s3://aws-emr-resources-678153616337-us-east-1/olist_classified_public_dataset_1/part-00000-aceb242a-0493-4f7a-b1d0-5f9c82fe151d-c000.csv

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
# order_products_value, order_status
# day_of_week, week_of_month
# customer_city
df4 = df3.groupBy('order_status', 'week_of_month', 'day_of_week', 'customer_city').count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
df4.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-------------+-----------+-----------------+-----+
|order_status|week_of_month|day_of_week|    customer_city|count|
+------------+-------------+-----------+-----------------+-----+
|   delivered|            5|          6|Sao Mateus do Sul|    3|
|   delivered|            2|          2|  Pindamonhangaba|    1|
|   delivered|            3|          1|         BRASILIA|    1|
|   delivered|            4|          3|         BRASILIA|    2|
|   delivered|            2|          5|  Duque de Caxias|    2|
|   delivered|            2|          2|Braganca Paulista|    1|
|   delivered|            1|          7|       Ananindeua|    1|
|   delivered|            5|          7|        Sao Paulo|    3|
|   delivered|            5|          4|        Arapiraca|    1|
|   delivered|            3|          3|           Canoas|    1|
|   delivered|            3|          1|           Sabara|    1|
|   delivered|            3|          4|             Maua|    1|
|   delivered|           

In [44]:
df4.write.format('csv').option('header','true').save('s3://aws-emr-resources-678153616337-us-east-1/olist_classified_public_dataset_2')

#'s3://aws-emr-resources-678153616337-us-east-1/olist_classified_public_dataset_2/part-00000-aceb242a-0493-4f7a-b1d0-5f9c82fe151d-c000.csv'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [96]:
average_review_score = df3.select(avg('review_score')).first()[0]

In [97]:
average_review_score

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3.5228794642857144

In [64]:
approved_order_count = df3.filter(col('order_aproved_at').isNotNull() ).count()
approved_order_count

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3584

In [92]:
total_order_value = df3.agg(sum('order_products_value')).first()[0]


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [93]:
total_order_value

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

477538.72000000544

In [95]:
print(f'Average order approval = {total_order_value/approved_order_count}')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Average order approval = 133.24183035714438

In [99]:
average_frieght_value = df3.select(avg('order_freight_value')).first()[0]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [100]:
average_frieght_value

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

22.40033761160717

In [110]:
df3.select(avg('review_score').alias('average_review_score'), 
           avg('order_freight_value').alias('average_frieght_value'), 
           avg('order_products_value').alias('Average_order_approval')) \
.first()[0:3]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(3.5228794642857144, 22.40033761160717, 133.24183035714438)

In [114]:
df5 = df3.withColumn('order_deliver_timestamp', to_timestamp(col('order_delivered_customer_date'))) \
        .withColumn('order_purchase_timestamp_conv', to_timestamp(col('order_purchase_timestamp')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [115]:
df5.select('order_deliver_timestamp', 'order_purchase_timestamp_conv').show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------+-----------------------------+
|order_deliver_timestamp|order_purchase_timestamp_conv|
+-----------------------+-----------------------------+
|   2017-09-08 20:35:...|          2017-08-30 11:41:01|
|   2017-09-29 21:13:...|          2017-09-26 09:13:36|
|   2018-01-23 17:51:...|          2018-01-15 15:50:42|
|   2018-02-20 19:38:...|          2018-02-04 11:16:42|
|   2017-12-19 22:33:...|          2017-12-07 11:58:42|
|   2017-11-09 19:09:...|          2017-11-02 11:33:03|
|   2017-12-19 14:13:...|          2017-11-26 23:58:24|
|   2017-12-26 16:09:...|          2017-12-08 21:13:04|
|   2017-07-10 16:35:...|          2017-07-04 16:12:36|
|   2017-12-18 15:22:...|          2017-11-29 04:25:45|
+-----------------------+-----------------------------+
only showing top 10 rows

In [125]:
df6 = df5.withColumn('deliver_time', 
               col('order_deliver_timestamp').cast('long')- 
               col('order_purchase_timestamp_conv').cast('long'))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [127]:
df6.select('deliver_time').first()
#df5.select(avg('deliver_time'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(deliver_time=809666)

In [162]:
average_delivery_time = df6.select(avg('deliver_time')).first()[0]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [161]:
total_frieght_charges = df.select(sum('order_freight_value')).first()[0]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [160]:
df.groupBy('customer_city').agg(sum('order_freight_value')).show()
    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------------+
|       customer_city|sum(order_freight_value)|
+--------------------+------------------------+
|Quilometro 14 do ...|                   16.11|
|    Francisco Morato|                   37.97|
|   Lagoinha do Piaui|                   34.61|
|           Americana|                  114.91|
|           Nova Lima|                  111.08|
|          Pedra Azul|                   16.28|
|        Francisco Sa|                    15.1|
|Sao Domingos do P...|                   12.75|
|             Fartura|                   29.89|
|       Sao Cristovao|                   21.15|
|             Pirajui|                   85.12|
|                Tupa|                   24.33|
|              PIRAPO|                   29.76|
|   Bom Jesus da Lapa|                   28.49|
| Santa Cruz de Minas|                    30.2|
|        Iracemapolis|                   12.96|
|             Messias|      166.79000000000002|
|           Campo Bom|                  

In [150]:
df.select('customer_city').distinct().count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

897

In [ ]:
df6.write.csv('s3://aws-emr-resources-678153616337-us-east-1/olist_classified_public_dataset_final')